In [1]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="wCRp7xFVwBmpykHBa7RQ")
# project = rf.workspace().project("construction-site-safety")
# model = project.version(27).model

# # infer on a local image
# print(model.predict("pm1.jpg", confidence=40, overlap=30).json())

# # visualize your prediction
# model.predict("pm1.jpg", confidence=40).save("prediction.jpg")

loading Roboflow workspace...
loading Roboflow project...
{'predictions': [{'x': 969, 'y': 494, 'width': 196, 'height': 423, 'confidence': 0.9051192998886108, 'class': 'Person', 'image_path': 'pm1.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 442, 'y': 560, 'width': 216, 'height': 480, 'confidence': 0.9003984928131104, 'class': 'Person', 'image_path': 'pm1.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 423, 'y': 393, 'width': 39, 'height': 26, 'confidence': 0.8963104486465454, 'class': 'NO-Mask', 'image_path': 'pm1.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 795, 'y': 334, 'width': 52, 'height': 37, 'confidence': 0.8711215257644653, 'class': 'Hardhat', 'image_path': 'pm1.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 554, 'y': 376, 'width': 39, 'height': 21, 'confidence': 0.8711177110671997, 'class': 'NO-Mask', 'image_path': 'pm1.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 823, 'y': 514, 'width': 192, 'height': 406, 'confidence': 0.86

In [2]:
from ultralytics import YOLO

model = YOLO("best.pt")
# print(model.predict("pm1.jpg")["boxes"])
results = model.predict("pm1.jpg")
# print(results['boxes'])
# import json 

# data = json.loads(results)

for result in results:
    # print(result)
    data = result.boxes  # Boxes object for bbox outputs
    # masks = result.masks  # Masks object for segmentation masks outputs
    # probs = result.probs  # Class probabilities for classification outputs
    print(data[0].)



image 1/1 D:\NCKH\Test\pm1.jpg: 448x640 4 Hardhats, 1 NO-Hardhat, 5 NO-Masks, 5 NO-Safety Vests, 5 Persons, 195.8ms
Speed: 6.6ms preprocess, 195.8ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[477.4365, 311.4075, 645.7624, 761.3195,   0.9020,   5.0000]])
cls: tensor([5.])
conf: tensor([0.9020])
data: tensor([[477.4365, 311.4075, 645.7624, 761.3195,   0.9020,   5.0000]])
id: None
is_track: False
orig_shape: tensor([ 932, 1398])
shape: torch.Size([1, 6])
xywh: tensor([[561.5994, 536.3635, 168.3259, 449.9120]])
xywhn: tensor([[0.4017, 0.5755, 0.1204, 0.4827]])
xyxy: tensor([[477.4365, 311.4075, 645.7624, 761.3195]])
xyxyn: tensor([[0.3415, 0.3341, 0.4619, 0.8169]])


In [22]:
import json
import cv2
import base64
import requests
import time

# Check to be sure your config file contains details for a Classification Model trained with Roboflow Train
# https://docs.roboflow.com/train | https://docs.roboflow.com/inference-classification/hosted-api
# load config file:
# with open('roboflow_config.json') as f:
#     config = json.load(f)

#     ROBOFLOW_API_KEY = config["ROBOFLOW_API_KEY"]
#     ROBOFLOW_SIZE = config["ROBOFLOW_SIZE"]
#     ROBOFLOW_MODEL_ID = config["ROBOFLOW_MODEL_ID"]
#     ROBOFLOW_VERSION_NUMBER = config["ROBOFLOW_VERSION_NUMBER"]

#     FRAMERATE = config["FRAMERATE"]
#     BUFFER = config["BUFFER"]

# # Construct the Roboflow Infer URL
# # (if running locally replace https://classify.roboflow.com/ with eg http://127.0.0.1:9001/)
# upload_url = "".join([
#     "https://classify.roboflow.com/",
#     ROBOFLOW_MODEL_ID, "/",
#     ROBOFLOW_VERSION_NUMBER,
#     "?api_key=",
#     ROBOFLOW_API_KEY,
#     "&format=json",
#     ])

# Get webcam interface via opencv-python
# Change '0' to '1' or '2' if it cannot find your webcam
video = cv2.VideoCapture(0)

# Infer via the Roboflow Infer API and return the result
def infer():
    # Get the current image from the webcam
    ret, img = video.read()

    # Resize (while maintaining the aspect ratio) to improve speed and save bandwidth
    height, width, channels = img.shape
    scale = 640 / max(height, width)
    img = cv2.resize(img, (round(scale * width), round(scale * height)))

    # Encode image to base64 string
    retval, buffer = cv2.imencode('.jpg', img)
    img_str = base64.b64encode(buffer)

    # Get prediction from Roboflow Infer API
    resp = requests.post(upload_url, data=img_str, headers={
        "Content-Type": "application/x-www-form-urlencoded"
    }, stream=True)
    # Convert Response to JSON
    predictions = resp.json()
    
    # Add predictions (class label and confidence score) to image
    (text_width, text_height), _ = cv2.getTextSize(
        f"{predictions['top']} Confidence: {predictions['confidence']}",
        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.6, thickness=2)
    
    text_location = (5, text_height)
    
    cv2.putText(img, f"{predictions['top']} | Confidence: {predictions['confidence']}",
                text_location, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.6,
                color=(255,255,255), thickness=2)   

    return img, predictions


# Main loop; infers sequentially until you press "q"
while 1:
    # On "q" keypress, exit
    if(cv2.waitKey(1) == ord('q')):
        break

    # Capture start time to calculate fps
    start = time.time()

    # Synchronously get a prediction from the Roboflow Infer API
    image, detections = infer()
    # And display the inference results
    cv2.imshow('image', image)

    # Print frames per second
    print((1/(time.time()-start)), "fps")
    print(detections)

# Release resources when finished
video.release()
cv2.destroyAllWindows()


KeyError: 'top'

In [4]:
import cv2
from ultralytics import YOLO

cap = cv2.VideoCapture("test.mp4")
model = YOLO("best.pt")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.track(frame, persist=True)
    print(results)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    for box, id in zip(boxes, ids):
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Id {id}",
            (box[0], box[1]),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
        )
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break


c:\Users\ADMIN\miniconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
0: 384x640 (no detections), 365.6ms
Speed: 3.1ms preprocess, 365.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'Hardhat', 1: 'Mask', 2: 'NO-Hardhat', 3: 'NO-Mask', 4: 'NO-Safety Vest', 5: 'Person', 6: 'Safety Cone', 7: 'Safety Vest', 8: 'machinery', 9: 'vehicle'}
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

      

AttributeError: 'NoneType' object has no attribute 'cpu'

In [3]:
from ultralytics import YOLO

model = YOLO("best.pt")  # or a segmentation model .i.e yolov8n-seg.pt
model.track(
    source="pm1.jpg"
)

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "c:\Users\ADMIN\miniconda3\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [5]:
from ultralytics import YOLO

model = YOLO("best.pt")

for result in model.track(source="test.mp4"):
    print(
        result.boxes.id.cpu().numpy().astype(int)
    )  # this will print the IDs of the tracked objects in the frame



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/842) D:\NCKH\Test\test.mp4: 384x640 (no detections), 264.8ms
video 1/1 (2/842) D:\NCKH\Test\test.mp4: 384x640 (no detections), 249.3ms
video 1/1 (3/842) D:\NCKH\Test\test.mp4: 384x640 (no detections), 233.1ms
video 1/1 (4/842) D:\NCKH\Test\test.mp4: 384x640 (no detections), 219.8ms
video 1/1 (5/842) D:\NCKH\Test\test.mp4: 384x640 (no detections), 227.0ms
video 1/1 (6/842) D:\NCKH\Test\test.mp4: 384x640 (no detections), 239.9ms
video 1/1 (7/842)

KeyboardInterrupt: 

In [2]:
from ultralytics import YOLO

model = YOLO('best.pt')
results = model.track(source=, conf=0.7, iou=0.7, show=True) 



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

c:\Users\ADMIN\miniconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
video 1/1 (1/842) D:\NCKH\Test\test.mp4: 384x640 (no detec

: 